## Reexecução

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.environ['SNORKELDB'] = 'postgres:///lzirondi'
from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
relation = candidate_subclass('Causa', ['Substância', 'Evento_Adverso'])

## Criando os arquivos dos dicionários

In [5]:
import Preprocessing as prep

inPath = ''
outPath = ''
corpusPath = ''
LIWCpath = '/home/lzirondi/Github/snorkel/Scripts/'

util = prep.Util(inPath, outPath, LIWCpath, False)

util.LIWKdicios()

Dicionário LIWC de Causas gerado com sucesso.
Dicionário LIWC de Verbos gerado com sucesso.
Dicionário LIWC de Causas & Verbos gerado com sucesso.


## Criando os dicionários

In [6]:
paths = ['/home/lzirondi/Github/snorkel/Scripts/Dicts/Causas',
         '/home/lzirondi/Github/snorkel/Scripts/Dicts/Verbos',
         '/home/lzirondi/Github/snorkel/Scripts/Dicts/Causas & Verbos'
]

causas = {}
verbos = {}
causas_e_verbos = {}

dicios = {
    paths[0] : causas,
    paths[1] : verbos,
    paths[2] : causas_e_verbos
}

for file in paths:
    f = open(file, 'r', encoding='UTF-8')
    s = f.readlines()
    f.close()
    
    for word in s:
        word = word.replace('\n', '')
        dicios.get(file)[word] = 0    

## Análise

In [ ]:
#%%time

from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)

from nltk.corpus import stopwords

cands = session.query(relation).filter(relation.split == 0).all()

sw = set(stopwords.words("portuguese"))

dicios = [causas, verbos, causas_e_verbos]

#chegar no candidato e ver se entre os tokens tem algo que a gente quer

for candidate in cands:
    for word in get_between_tokens(candidate):
        if word not in sw:
            print(word)
            aux = False
            if causas.get(word):
                causas[word] = causas[word] + 1
                print(word)
                if verbos.get(word):
                    aux = True
                    verbos[word] = verbos[word] + 1
                    causas_e_verbos[word] = causas_e_verbos[word] + 1

            if(aux):
                print(word)
                verbos[word] = verbos[word] + 1

In [50]:
#%%time

from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)

from nltk.corpus import stopwords


import re

numero = re.compile('\D+')
repeticao = re.compile('\w{2,}')

cands = session.query(relation).filter(relation.split == 0).all()

sw = set(stopwords.words("portuguese"))

dic = {}

#nao repetir frases
#ignorar palavras de 1 letra 

for candidate in cands:
    for word in get_between_tokens(candidate):
        if word not in sw and len(word)>1 and numero.match(word):
            if dic.get(word):
                dic[word] = dic[word] + 1
            else:
                dic[word] = 1
                
ordenada = []
maior = ['', 0]

dici = dic


for i in range(0, len(dici)):
    for key in dici:
        if dici[key] > maior[1]:
            maior[0] = key
            maior[1] = dici[key]
    ordenada.append(maior)
    dici.pop(maior[0])
    maior = ['', 0]
                

f = open('/home/lzirondi/Github/snorkel/Scripts/Análise de Palavras/between_tokens_stopFiltered.tsv', 'w', encoding='UTF-8')

for word in ordenada:
    f.write(word[0] + '\t' + str(word[1]) + '\n')
    
f.close()

## Testes

In [ ]:
cands = session.query(relation).filter(relation.split == 0).all()

from snorkel.models import Document

docs = session.query(Document).order_by(Document.name).all()
#print(session.query(relation).filter(relation.split == 1))


#candi = cands[0]

print(type(candi))
print(cands[0].id)
for candi in cands:
    print(candi.id)

In [41]:
from snorkel.models import Document, Sentence

print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

print(len(cands))

rep = []

for candidate in cands:
    s = candidate.get_parent().stable_id.split(':')[0]
    
    if s not in rep:
        rep.append(candidate.id)

print(len(rep))
print(len(cands))
#print(dir(cands[0]))
#print(cands[0].get_parent())

print(cands[0].get_parent().stable_id)


Documents: 5984
Sentences: 6031
6398
6398
6398
3099::sentence:0:138
